## 1.a) Extraction

In [ ]:
#Importing libraries

import pandas as pd 
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt 
import requests

print("Libraries imported successfully......")

The link to all these APIs is: https://www.game-change.co.uk/2023/02/10/a-complete-guide-to-the-fantasy-premier-league-fpl-api/

Descriptions of each dataframe are in the notes.

In [ ]:
# Imported full data from last seasons

url = 'https://fantasy.premierleague.com/api/bootstrap-static/'
r = requests.get(url)
json = r.json()


#Convert to df

elements_df = pd.DataFrame(json['elements'])
elements_types_df = pd.DataFrame(json['element_types'])
teams_df = pd.DataFrame(json['teams'])

In [ ]:
### This cell imports data from any gameweek of the current season as long as the gameweek has ended or is live.
gameweek_url = 'https://fantasy.premierleague.com/api/event/2/live/'
r = requests.get(gameweek_url)
gameweek_json = r.json()

gameweek_df = pd.DataFrame(gameweek_json['elements'])


In [79]:
### Function to return player url 
def format_player_url(player_id):
    # Define the base URL without curly brackets
    base_url = 'https://fantasy.premierleague.com/api/element-summary/{}/'
    
    # Replace the placeholder '{}' with the actual player_id
    formatted_url = base_url.format(player_id)
    
    return formatted_url

# Example usage:
player_id = 308  # Replace this with the actual player ID
formatted_url = format_player_url(player_id)
print(formatted_url)


https://fantasy.premierleague.com/api/element-summary/308/


In [83]:
#This cell imports individual player information
r = requests.get(format_player_url(308))
player_json = r.json()


player_fixtures_df = pd.DataFrame(player_json['fixtures'])
player_history_df = pd.DataFrame(player_json['history'])
player_history_past_df = pd.DataFrame(player_json['history_past'])


In [ ]:
#This cell imports individual player information
fixture_url = 'https://fantasy.premierleague.com/api/fixtures/?event=1'
r = requests.get(fixture_url)
fixture_json = r.json()



In [ ]:
fixture_df = pd.DataFrame(fixture_json)


In [ ]:
fixture_df

In [ ]:
elements_df.loc[:, ['id', 'web_name','news','form', 'team_code', 'element_type']].sort_values('form' ,ascending = False)

In [ ]:
player_history_df.loc[:,['element','fixture','opponent_team','round', 'influence', 'creativity', 'threat', 'ict_index']]

In [ ]:
player_history_usable = player_history_df.loc[:,['element', 'fixture', 'opponent_team', 'was_home','round']]

In [ ]:
player_history_usable

In [ ]:
player_history_past_usable = player_history_past_df.loc[:,['season_name', 'element_code','total_points','influence', 'creativity','threat', 'ict_index','expected_goals', 'expected_assists','expected_goal_involvements', 'expected_goals_conceded']]

In [ ]:
elements_usable = elements_df.loc[:,['chance_of_playing_next_round', 'chance_of_playing_this_round', 'code','element_type', 
        'first_name', 'form', 'id','points_per_game',
        'second_name', 'selected_by_percent', 'special',
        'web_name','expected_goals', 'expected_assists',
        'expected_goal_involvements', 'expected_goals_conceded',
        'expected_goals_per_90', 'saves_per_90', 'expected_assists_per_90',
        'expected_goal_involvements_per_90', 'expected_goals_conceded_per_90',
        'goals_conceded_per_90','starts_per_90', 'clean_sheets_per_90']]

In [ ]:

# Convert 'form' column to float
elements_usable['form'] = elements_usable['form'].astype(float)


elements_usable = elements_usable[elements_usable['form'] != 0.0]



In [ ]:
elements_usable

In [ ]:
mapping_df = elements_usable[['element_type', 
        'first_name', 'form', 'id','points_per_game',
        'second_name', 'selected_by_percent', 'special',
        'web_name','expected_goals', 'expected_assists',
        'expected_goal_involvements', 'expected_goals_conceded',
        'expected_goals_per_90', 'saves_per_90', 'expected_assists_per_90',
        'expected_goal_involvements_per_90', 'expected_goals_conceded_per_90',
        'goals_conceded_per_90','starts_per_90', 'clean_sheets_per_90']].drop_duplicates()

# Print the modified DataFrame



In [ ]:
mapping_df.to_csv('constant_mapping.csv', index=False)



In [ ]:
# Load the mapping DataFrame
constant_mapping = pd.read_csv('constant_mapping.csv')

# Merge the mapping DataFrame with your new DataFrame (e.g., df_new)
gameweek_df = gameweek_df.merge(constant_mapping, on='id', how='left')

In [ ]:
gameweek_df

In [ ]:
# Load the mapping DataFrame
constant_mapping = pd.read_csv('constant_mapping.csv')

# Merge the mapping DataFrame with your new DataFrame (e.g., df_new)
df_new = df_new.merge(constant_mapping, on='ID', how='left')


## 1.b) Loading


In [ ]:
player_history_df